In [1]:
from neuralprophet import NBeats, LSTM, DeepAR, NeuralProphet, TFT
from neuralprophet.utils.df_utils import split_df


import pandas as pd
import numpy as np
import os

In [2]:
from neuralprophet.tools.metrics_libra import *

In [3]:
data_loc = '../example_data/LIBRA/'
datasets = os.listdir(data_loc)


datasets_economics = [dataset for dataset in datasets if 'economics' in dataset]
datasets_finance = [dataset for dataset in datasets if 'finance' in dataset]
datasets_human = [dataset for dataset in datasets if 'human' in dataset]
datasets_nature = [dataset for dataset in datasets if 'nature' in dataset]

In [4]:
usecase = 'nature'

datasets = {}

if usecase == 'economics':
    for dataset in datasets_economics:
        datasets.update({dataset:pd.read_csv(data_loc + dataset)})
elif usecase == 'finance':
    for dataset in datasets_finance:
        datasets.update({dataset:pd.read_csv(data_loc + dataset)})
elif usecase == 'human':
    for dataset in datasets_human:
        datasets.update({dataset:pd.read_csv(data_loc + dataset)})
elif usecase == 'nature':
    for dataset in datasets_nature:
        datasets.update({dataset:pd.read_csv(data_loc + dataset)})
    
    
frequencies = pd.read_csv(data_loc + 'freq.csv')
mapping_frequencies_economics = {
    1 : 'D',
    4: 'Q',
    12: 'M',
    52: 'W',
    24: 'H',
    7: 'W',
    91: 'D',
    364: 'D', 
    360: 'D',
    168: 'D',
    672: 'D',
    96: 'H', 
    288: 'D',
    28: 'D',
    6: 'D',
    30: 'D',
    720: 'D'
    
}

def mapping(x):
    try:
        return mapping_frequencies_economics[x]
    except:
        return 'D'


Following the original LIBRA work

# One step ahead

In [5]:
valid_p = 0.2
n_forecasts = 1

In [ ]:

metrics = {}

counter = 0
for dataset_name, df in datasets.items():
    one_dataset_metrics = {}

    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]

    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    
    
    n_epochs = 50
    if len(df) == 372864:
        freq = 'H'
        n_epochs = 15
    
    n_lags = freq_number
    
    method = 'LSTM_1step'
    m = LSTM(n_lags = n_lags,
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=n_epochs)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    one_dataset_metrics.update({
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    })
    
    
    method = 'NP_1step'
    m = NeuralProphet(n_lags = n_lags,
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=n_epochs)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    one_dataset_metrics.update({
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    })
    
    
    method = 'DeepAR_1step'
    m = DeepAR(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=n_epochs)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    one_dataset_metrics.update({
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    })
    
    method = 'NBeats_1step'
    m = NBeats(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=n_epochs)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    one_dataset_metrics.update({
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    })
    
    method = 'TFT_1step'
    m = TFT(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=n_epochs)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    one_dataset_metrics.update({
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    })
    
    metrics.update({dataset_name:one_dataset_metrics})
    
    counter += 1

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16


  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 960   
1 | linear    | Linear       | 11    
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
971       Trainable params
0         Non-trainable params
971       Total params
0.004     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/fds/neural_prophet/neuralprophet/tools/metrics_libra.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  return y_pred_mae / naive_mae

INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16





  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name          | Type          | Params
----

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [8]:
pd.DataFrame(metrics)

,nature_83.csv,nature_97.csv,nature_68.csv,nature_40.csv
smape_TFT_1step,28.105743,14.503072,1.670347,NaN
mase_TFT_1step,inf,3.749676,2.978227,NaN
mues_TFT_1step,0.639535,0.666667,0.363636,NaN
moes_TFT_1step,0.360465,0.333333,0.636364,NaN
muas_TFT_1step,0.156907,0.086928,0.006151,NaN
moas_TFT_1step,0.115789,0.056377,0.010603,NaN
smape_NP_1step,NaN,NaN,NaN,33.813974
mase_NP_1step,NaN,NaN,NaN,inf
mues_NP_1step,NaN,NaN,NaN,0.368128
moes_NP_1step,NaN,NaN,NaN,0.631872


# Multi step ahead

In [ ]:
valid_p = 0.2
n_forecasts_p = 0.2

In [ ]:

metrics = {}

counter = 0
for dataset_name, df in datasets.items():

    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]

    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    
    n_lags = freq_number
    n_forecasts = int(n_forecasts_p*len(df))
    
    method = 'LSTM_1step'
    m = LSTM(n_lags = n_lags,
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=50)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    }})
    
    
    method = 'NP_1step'
    m = NeuralProphet(n_lags = n_lags,
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=50)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    }})
    
    
    method = 'DeepAR_1step'
    m = DeepAR(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=50)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    }})
    
    method = 'NBeats_1step'
    m = NBeats(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=50)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    }})
    
    method = 'TFT_1step'
    m = TFT(n_lags = int(n_lags),
            n_forecasts=n_forecasts,
            learning_rate=0.01,
            epochs=50)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        f'smape_{method}': smapes,
        f'mase_{method}': mases,
        f'mues_{method}': mueses,
        f'moes_{method}': moeses,
        f'muas_{method}': muases,
        f'moas_{method}': moases
        
    }})
    
    
    
    counter += 1

In [ ]:
pd.DataFrame(metrics)